
#1.0 - Tabela Utilizada no Drive
Importando a última tabela disponibilizada pela MEDICINA USP.<br>
Nome da Tabela: NEW-BDIPMama-INTELI(1).csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.1 - Buscando a tabela e importando as bibliotecas utilizadas

In [2]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, plot_confusion_matrix
import numpy as np
import imblearn as imb

Nomeei a tabela como tabela_completa

In [3]:
tabela_completa = pd.read_csv ('/content/drive/MyDrive/DRIVE - USPMedicina/NEW-BDIPMamaV11-INTELI (3).csv',delimiter=";")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,24,25,75,76,77,78,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 2.0 - Observando os dados válidos
Para cada coluna da tabela, consultamos com o value_counts(), afim de consultar a quantidade de dados nas colunas, sendo assim, há a consulta para uma primeira seleção da tabela.<br><br>
Importante destacar que todas as colunas selecionadas até o momento tem mais de 1/3 dos registros válidos.
O restante foi ignorado devido à ausência de registros significativos.

Exemplificando com a coluna "redcap_repeat_instrument" do banco de dados

In [4]:
tabela_completa['redcap_repeat_instrument'].value_counts()

dados_antropometricos          50137
dados_histopatologicos_mama     4626
registro_de_tumores             4525
Name: redcap_repeat_instrument, dtype: int64

# 3.0 - Premissas assumidas

Assumimos que todos os NaN acima do primeiro dado numérico significa que o dado não foi preenchido no cadastro do paciente, sendo assim, transformaremos este dado nulo em 0.0<br>
Para não perder todos os dados da amostra foi preferível substituir os nulos (Nan) pelo número 0.0 até chegar o primeiro exposto na coluna, este foi replicado na coluna abaixo até encontrar o próximo valor válido, segundo orientações docente para o entendimento dos dados. A partir deste momento, lê-se a tabela da seguinte forma:<br>

*   Caso observe uma coluna onde o primeiro dado é -1, significa que este é Nan.

*   Caso encontre uma coluna onde o primeiro dado é 0, este dado é lido como Não.

Optamos por renomear a tabela completa, que está no drive, para tabela tratada, a fim de conseguir fazer consultas na tabela completa ao longo do projeto, sem prejudicar a modelagem dos dados que ocorre simultaneamente.<br><br>

Utilizamos o comando fillna para buscar na coluna um valor válido e repetir nos dados vazios que iriam se repetir normalmente para o mesmo record_id. Posteriormente,a fim de transformar as string dessa coluna em novas colunas com dados numéricos, especificamente, dados de afirmação (Sim) ou negação (Não), utilizamos o método dummies.

Sendo assim, lê-se a tabela da seguinte forma:

*   Caso observe uma coluna onde o primeiro dado é -1, significa que este é **Nan**;
*   Caso observe uma coluna onde o primeiro dado é 0, significa que este é lido como **Não**;
*   Caso Observe uma coluna onde o primeiro dado é 1, significa que este é lido como **Sim**;

Fizemos este comando para a coluna "redcap_repeat_instrument" visando replicar a coluna em três dados diferentes, pegando seus respectivos valores e transformando em colunas.

In [5]:
tabela_trat = tabela_completa.fillna(method='ffill').dropna(subset=['redcap_repeat_instrument']) #Utilizei o fillna com o método ffill para buscar o último valor válido e repetir na linha posterior.
tabela_trat['redcap_repeat_instrument'].unique()

tabela_tratada = pd.get_dummies(tabela_trat, columns=['redcap_repeat_instrument', 'antec_fam_cancer_mama', 'ultinfo']) #Utilizei o método get_dummies na coluna redcap_repeat_instrument para separar o conteúdo dela em novas colunas no final

A saída esperada é a exclusão da coluna "redcap_repeat_instrument", e a criação de três novas colunas, respectivamente:
* "redcap_repeat_instrument_dados_antropometricos";
* "redcap_repeat_instrument_dados_histopatologicos_mama"; e 
* "redcap_repeat_instrument_registro_de_tumores".

Sendo assim, indicando 1 e 0 nos registros do paciente.<br>

Fiz o mesmo processo para a coluna "antec_fam_cancer_mama" que indica se há algum familiar com câncer ou não, buscando transformar as string "Sim" e "Não" em 0 e 1.

Selecionamos as colunas que iremos utilizar, como retiramos as duplicatas com base 

In [6]:
ta_final = tabela_tratada[['record_id','t','n','m', 'ultinfo_1.0',	'ultinfo_2.0',	'ultinfo_3.0',	'ultinfo_4.0','breast_feeding_time', 'dob','tumor_subtype',
                           'antec_fam_cancer_mama_Não','antec_fam_cancer_mama_Sim', 'tempo_rep_hormo','bmi', 'follow_up_days', 'menarche', 'tobaco','alcohol',
                           'date_last_fu','familial_degree___1','familial_degree___2','familial_degree___3', 'primary_diganosis','benign']]
tabela_final = ta_final.fillna(-1)
tabela_final = tabela_final.drop_duplicates(subset='record_id', keep='last', ignore_index=True)

# 4.0 - Transformando Colunas
Foi necessário transformar a coluna data de nascimento (dob) em idade, visto que será mais fácil saber a idade da paciente quando gerarmos o modelo de predição.<br><br>
A data de nascimento foi calculada até a última consulta do mesmo, e não até a data presente. Preferimos demonstrar desta forma devido a não existência da data de óbito no banco de dados, e também, por nos concentrarmos em encontrar a relação da sobrevida do paciente até a ultima consulta.
<br><br>
Abaixo o código que gera a idade a partir da data de nascimento.

In [7]:
from datetime import datetime, date
tabela_final['dob']
def pegar_idade(nascimento, _date):
  nascimento = datetime.strptime(nascimento,"%d/%m/%Y").date()
  _date = datetime.strptime(_date,"%d/%m/%Y").date()
  idade = _date.year - nascimento.year
  if(nascimento.month == _date.month and nascimento.day >= _date.year):
    idade = idade + 1
  elif(nascimento.month > _date.month):
    idade = idade + 1
  return idade

def convertDate(age):
  return datetime.strptime(age,"%d/%m/%Y").date()
tabela_final['dob'].apply(convertDate)
tabela_final['date_last_fu'].apply(convertDate)
tabela_final['idade'] = ""
for ind in tabela_final.index:
    tabela_final['idade'][ind] = pegar_idade(tabela_final['dob'][ind], tabela_final['date_last_fu'][ind])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Com o intuito de realizar testes por meio de modelos de classificação, foi necessário transformar a coluna dos dias de sobrevida em um filtro, sendo adotado para essa aplicação uma rotulação de 1 a 4 com base em quartis de modo que os dados pudessem ser melhor enquadrados dentro da aplicação do modelo e evitasse ruídos nos resultados. A seguir, segue o código da função responsável por essa conversão:

In [8]:
days_since_the_last_tumor = tabela_final.follow_up_days.values
tabela_final['class_sobrevida'] = np.nan

lower_day = min(days_since_the_last_tumor)
max_day = max(days_since_the_last_tumor)


quartile = (max_day - lower_day)/4
first_quartile = lower_day + quartile
second_quartile = lower_day + (2*quartile)
third_quartile = lower_day + (3*quartile)

i = 0

for day in days_since_the_last_tumor:
  if day == 0:
    tabela_final.loc[i, 'class_sobrevida'] = -1
  if day <= first_quartile:
    tabela_final.loc[i, 'class_sobrevida'] = 1
  elif day <= second_quartile:
    tabela_final.loc[i, 'class_sobrevida'] = 2
  elif day <= third_quartile:
    tabela_final.loc[i, 'class_sobrevida'] = 3
  elif day > third_quartile:
    tabela_final.loc[i, 'class_sobrevida'] = 4
  else:
    tabela_final.loc[i, 'class_sobrevida'] = -1
  
  i += 1

Após essas alterações e a finalização da manipulação das colunas que recebem dados em formatos de data (dd/mm/aaaa), foi necessário eliminar essas colunas para que essa formatação de dados não interferisse como um ruído no treinamento e aprendizado do modelo.

In [9]:
tabela_final.pop('dob')
tabela_final.pop('date_last_fu')

0       01/01/2021
1       26/04/2014
2       17/11/2016
3       02/05/2019
4       24/05/2017
           ...    
4129    03/04/2021
4130    22/06/2021
4131    11/08/2021
4132    10/10/2021
4133    13/05/2021
Name: date_last_fu, Length: 4134, dtype: object

A seguir foi realizada apenas uma visualização da variável de saída do modelo para compreender como os dados estavam distribuídos dentro dessa coluna.

In [10]:
tabela_final['class_sobrevida'].value_counts()

2.0    1935
1.0    1432
3.0     623
4.0     144
Name: class_sobrevida, dtype: int64

Com a finalidade de facilitar a visualização e compreensão das colunas utilizadas no modelo, foi necessário renomear algumas Features, que podem ser visualizadas no código abaixo:

In [11]:
tabela_final.rename({'ultinfo_1.0':'ultinfo_1','ultinfo_2.0':'ultinfo_2',
                     'ultinfo_3.0':'ultinfo_3', 'ultinfo_4.0':'ultinfo_4',
                     'record_id':'pacientes', 't':'tumor', 'n':'nodulo',
                     'm':'metastese', 'breast_feeding_time':'tempo_de_amamentacao',
                     'dob':'data_de_nascimento', 'tobaco':'fumante', 'alcohol':'alcolatra',
                     'date_last_fu':'data_ultima_consulta', 'familial_degree___1':'grau_parentesco_1',
                     'familial_degree___2':'grau_parentesco_2', 'familial_degree___3':'grau_parentesco_3',
                     'primary_diganosis':'tipo_tumor', 'tumor_subtype':'subtipo_tumoral',
                     'menarche':'primeira_menstruacao', 'bmi':'IMC', 
                     'benign':'benigno/maligno'}, axis='columns', inplace=True)

Na célula abaixo, foi realizada a exportação da tabela já tratada no formato csv, que será usada futuramente para o treinamento e teste do modelos preditivos.

In [12]:
#exportação da tabela manipulada para um arquivo .xlsx (excel)
! pip install openpyxl
tabela_final.to_csv('tabela_tratada.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
